In [35]:
import numpy as np

from openneuron.units import Neuron
from openneuron.activations import *
from openneuron.losses import *
from openneuron.optimizers import *
from openneuron.initializers import *
from openneuron.utils import *

In [36]:
''' # Класс перенесен в units.py
# Класс для нейрона
class Neuron:
    count = 0

    def __init__(self, weights=HeUniform(), bias=HeUniform(), input_size=2, output_size=1, activation=linear, loss=MSE(), optimizer=SGD(momentum=0)):
        Neuron.count += 1
        self.number = Neuron.count  # номер нейрона
        self.id = id(self)
        self.layer = None  # ссылка на слой в котором находится нейрон
        self.index = None  # индекс нейрона в слое
        self.X = None
        self.weights = np.array([weights]).T if isinstance(weights, list) else weights(shape=(input_size, output_size))
        self.bias = np.array([[bias]]).T if isinstance(bias, (float, int)) else bias(shape=(1, output_size))
        self.activation = activation or linear  # функция активации нейрона
        self.loss_function = loss or MSE()
        self.optimizer = optimizer or SGD(momentum=0)
        self.gradient = {'weights': None, 'bias': None}

    @property
    def Inputs(self):
        if self.layer is not None:
            return self.layer.Inputs
        return self.X
    
    @Inputs.setter
    def Inputs(self, X):
        self.X = X

    @property
    def inputs(self):
        return self.Inputs[-1] if self.Inputs is not None else None

    @property
    def Z(self):
        if self.layer is not None:
            return self.layer.Z.T[self.index]
        return np.dot(self.Inputs, self.weights) + (self.bias.item() if self.bias.item() is not None else 0) if self.Inputs is not None else None

    @property
    def z(self):
        return self.Z[-1].item() if self.Z is not None else None
    
    @property
    def A(self):
        if self.layer is not None:
            return self.layer.A.T[self.index]
        return self.activation(self.Z) if self.Z is not None else None

    @property
    def a(self):
        return self.A[-1].item() if self.A is not None else None
    
    def activation_derivative(self, Z):
        return self.activation(Z, derivative=True)
    
    def forward(self, Inputs, training=True):
        self.Inputs = Inputs
        return self.A

    def backward(self, error):
        self.delta = self.loss_function.loss_derivative(error) * self.activation_derivative(self.Z)
        self.gradient['weights'] = np.dot(self.Inputs.T, self.delta)
        self.gradient['bias'] = np.sum(self.delta, axis=0)
        # return np.dot(self.delta, self.weights.T)  # delta ошибки передаваемая в следующий слой

    def update(self):
        self.optimizer.update(self)

    def fit(self, X, y, epochs=10, batch_size=1, final_batch_size=None, shuffle=True, validation_data=None):
        self.X = X
        self.epochs = epochs
        self.batch_size = batch_size
        self.final_batch_size = final_batch_size
        
        if validation_data is not None:
            X_test, y_test = validation_data
            val_data_type = 'Test Data'
        else:
            X_test, y_test = X, y
            val_data_type = 'Train Data'
        
        val_loss = self.loss_function.evaluate_loss(y_test, self.predict(X_test))
        print(f'Training started with Overall Loss {val_loss:.4f} on {val_data_type}')
        
        self.X_len = X.shape[0]
        for epoch in range(epochs):
            if self.batch_size is not None and self.final_batch_size is not None:
                batch_size = self.batch_size + int((self.final_batch_size - self.batch_size) * (epoch + 1) / epochs)
            
            if shuffle:
                permutation = np.random.permutation(self.X_len)
                X_shuffled = X[permutation]
                y_shuffled = y[permutation]
            else:
                X_shuffled = X
                y_shuffled = y
            
            epoch_loss = 0
            num_batches = 0
            
            for i in range(0, self.X_len, (batch_size or self.X_len)):
                num_batches += 1
                X_batch = X_shuffled[i:i+(batch_size or self.X_len)]
                y_batch = y_shuffled[i:i+(batch_size or self.X_len)]
                
                predictions_batch = self.forward(X_batch)
                error = self.loss_function.evaluate_error(y_batch, predictions_batch)
                loss = self.loss_function.evaluate_loss(y_batch, predictions_batch)
                epoch_loss += loss
                
                self.backward(error)
                self.update()
            
            loss = epoch_loss / num_batches
            val_loss = self.loss_function.evaluate_loss(y_test, self.predict(X_test))
            
            # Выводим на новой строке только каждую 10-ю эпоху
            end = '\n' if (epochs < 10 or epoch == 0 or (epoch + 1) % (epochs // 10) == 0 or epoch == epochs - 1) else '\r' 
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f} on {val_data_type}', end=end)
        print(f'Training completed with Overall Loss {val_loss:.4f} on {val_data_type}')

    def predict(self, X):
        return self.forward(X, training=False)
    
    def call(self, X, training=False):
        """ # Метод call вызывается после выполнения встроенного метода __call__.
        Служит для дополнения/изменения работы встроенного метода __call__ не нарушая необходимый для правильной работы процесс
        вычисления данных при вызове объекта. На вход получает входные данные (массив X). По умолчанию возвращает значение(я) 
        активированного выхода нейрона (массив A).
        """
        # Любой дополнительный код
        return self.A

    def __call__(self, Inputs, training=False):
        self.forward(Inputs, training)
        return self.call(Inputs, training)

    def __str__(self):
        return f'Neuron {format(self.number, decimals=decimals)}, inputs: {format(self.inputs, decimals=decimals)}, ' \
               f'weights: {format(self.weights.flatten(), decimals=decimals)}, bias: {format(self.bias.item(), decimals=decimals)}, ' \
               f'z: {format(self.z, decimals=decimals)}, activation: {self.activation.__name__}, a: {format(self.a, decimals=decimals)}'
'''

' # Класс перенесен в units.py\n# Класс для нейрона\nclass Neuron:\n    count = 0\n\n    def __init__(self, weights=HeUniform(), bias=HeUniform(), input_size=2, output_size=1, activation=linear, loss=MSE(), optimizer=SGD(momentum=0)):\n        Neuron.count += 1\n        self.number = Neuron.count  # номер нейрона\n        self.id = id(self)\n        self.layer = None  # ссылка на слой в котором находится нейрон\n        self.index = None  # индекс нейрона в слое\n        self.X = None\n        self.weights = np.array([weights]).T if isinstance(weights, list) else weights(shape=(input_size, output_size))\n        self.bias = np.array([[bias]]).T if isinstance(bias, (float, int)) else bias(shape=(1, output_size))\n        self.activation = activation or linear  # функция активации нейрона\n        self.loss_function = loss or MSE()\n        self.optimizer = optimizer or SGD(momentum=0)\n        self.gradient = {\'weights\': None, \'bias\': None}\n\n    @property\n    def Inputs(self):\n 

In [37]:
# Пример AND
# Настройка вывода для удобочитаемости
np.set_printoptions(precision=4, suppress=True, threshold=6, edgeitems=2, linewidth=80)

X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([[0],
              [0],
              [0],
              [1]])

X_train, y_train = X, y
X_test, y_test = X, y

neuron = Neuron(weights=[0.45, -0.12], bias=1.0, activation=sigmoid, loss=MSE(), optimizer=SGD(learning_rate=0.7, momentum=0.3))
neuron.fit(X_train, y_train, epochs=100, shuffle=False)

# y_pred = neuron.predict(X_test)
y_pred = neuron(X_test)
print(f'\nPredictions:\n{y_pred}')
print(f'\nLast forward:\n{neuron}')

Training started with Overall Loss 0.4335 on Train Data
Epoch 1/100, Loss: 0.4319, Validation Loss: 0.3012 on Train Data
Epoch 10/100, Loss: 0.1408, Validation Loss: 0.1109 on Train Data
Epoch 20/100, Loss: 0.0828, Validation Loss: 0.0677 on Train Data
Epoch 30/100, Loss: 0.0566, Validation Loss: 0.0481 on Train Data
Epoch 40/100, Loss: 0.0422, Validation Loss: 0.0369 on Train Data
Epoch 50/100, Loss: 0.0333, Validation Loss: 0.0297 on Train Data
Epoch 60/100, Loss: 0.0273, Validation Loss: 0.0247 on Train Data
Epoch 70/100, Loss: 0.0230, Validation Loss: 0.0211 on Train Data
Epoch 80/100, Loss: 0.0198, Validation Loss: 0.0183 on Train Data
Epoch 90/100, Loss: 0.0174, Validation Loss: 0.0162 on Train Data
Epoch 100/100, Loss: 0.0154, Validation Loss: 0.0145 on Train Data
Training completed with Overall Loss 0.0145 on Train Data

Predictions:
[[0.0041]
 [0.1311]
 [0.1338]
 [0.8492]]

Last forward:
Neuron, inputs: [1 1], weights: [3.62 3.6], bias: -5.49, z: 1.73, activation: sigmoid, a: 

In [38]:
# Пример простой регрессии
# Настройка вывода для удобочитаемости
np.set_printoptions(precision=4, suppress=True, threshold=6, edgeitems=2, linewidth=80)

X = np.array([[1, 0]])
y = np.array([[1]])

X_train, y_train = X, y
X_test, y_test = X, y

neuron = Neuron(activation=sigmoid, loss=MSE(), optimizer=Adam(learning_rate=1))
neuron.fit(X_train, y_train, epochs=10)

# y_pred = neuron.predict(X_test)
y_pred = neuron(X_test)
print(f'\nPredictions:\n{y_pred}')
print(f'\nLast forward:\n{neuron}')

Training started with Overall Loss 0.5697 on Train Data
Epoch 1/10, Loss: 0.5697, Validation Loss: 0.3851 on Train Data
Epoch 2/10, Loss: 0.3851, Validation Loss: 0.2552 on Train Data
Epoch 3/10, Loss: 0.2552, Validation Loss: 0.1647 on Train Data
Epoch 4/10, Loss: 0.1647, Validation Loss: 0.1044 on Train Data
Epoch 5/10, Loss: 0.1044, Validation Loss: 0.0659 on Train Data
Epoch 6/10, Loss: 0.0659, Validation Loss: 0.0419 on Train Data
Epoch 7/10, Loss: 0.0419, Validation Loss: 0.0271 on Train Data
Epoch 8/10, Loss: 0.0271, Validation Loss: 0.0179 on Train Data
Epoch 9/10, Loss: 0.0179, Validation Loss: 0.0122 on Train Data
Epoch 10/10, Loss: 0.0122, Validation Loss: 0.0085 on Train Data
Training completed with Overall Loss 0.0085 on Train Data

Predictions:
[[0.9078]]

Last forward:
Neuron, inputs: [1 0], weights: [0.65 0.9], bias: 1.64, z: 2.29, activation: sigmoid, a: 0.91


In [39]:
import tensorflow as tf


initializer = tf.keras.initializers.GlorotNormal()
values = initializer(shape=(4, 2))
print(values)

tf.Tensor(
[[-0.164  -0.0596]
 [ 0.2807  0.1232]
 [-0.0974  0.2396]
 [ 0.7205  0.7953]], shape=(4, 2), dtype=float32)


In [40]:
# Пример AND
# Настройка вывода для удобочитаемости
np.set_printoptions(precision=4, suppress=True, threshold=6, edgeitems=2, linewidth=80)

X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([[0],
              [0],
              [0],
              [1]])

X_train, y_train = X, y
X_test, y_test = X, y

neuron = Neuron(activation=sigmoid, optimizer=Adam(learning_rate=1))
neuron.fit(X_train, y_train, epochs=25)

# y_pred = neuron.predict(X_test)
y_pred = neuron(X_test)
print(f'\nPredictions:\n{y_pred}')
print(f'\nLast forward:\n{neuron}')

Training started with Overall Loss 0.3786 on Train Data
Epoch 1/25, Loss: 0.3193, Validation Loss: 0.2017 on Train Data
Epoch 2/25, Loss: 0.1989, Validation Loss: 0.1149 on Train Data
Epoch 4/25, Loss: 0.1226, Validation Loss: 0.1125 on Train Data
Epoch 6/25, Loss: 0.1007, Validation Loss: 0.0932 on Train Data
Epoch 8/25, Loss: 0.0877, Validation Loss: 0.0801 on Train Data
Epoch 10/25, Loss: 0.0777, Validation Loss: 0.0730 on Train Data
Epoch 12/25, Loss: 0.0682, Validation Loss: 0.0632 on Train Data
Epoch 14/25, Loss: 0.0618, Validation Loss: 0.0569 on Train Data
Epoch 16/25, Loss: 0.0552, Validation Loss: 0.0496 on Train Data
Epoch 18/25, Loss: 0.0472, Validation Loss: 0.0440 on Train Data
Epoch 20/25, Loss: 0.0440, Validation Loss: 0.0388 on Train Data
Epoch 22/25, Loss: 0.0391, Validation Loss: 0.0347 on Train Data
Epoch 24/25, Loss: 0.0340, Validation Loss: 0.0313 on Train Data
Epoch 25/25, Loss: 0.0315, Validation Loss: 0.0290 on Train Data
Training completed with Overall Loss 0.

In [41]:
# Пример OR
# Настройка вывода для удобочитаемости
np.set_printoptions(precision=4, suppress=True, threshold=6, edgeitems=2, linewidth=80)

X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([[0],
              [1],
              [1],
              [1]])

X_train, y_train = X, y
X_test, y_test = X, y

neuron = Neuron(activation=heaviside, loss=MAE(), optimizer=SGD(learning_rate=1, momentum=0))
neuron.fit(X_train, y_train, epochs=5)

# y_pred = neuron.predict(X_test)
y_pred = neuron(X_test)
print(f'\nPredictions:\n{y_pred}')
print(f'\nLast forward:\n{neuron}')

Training started with Overall Loss 0.7500 on Train Data
Epoch 1/5, Loss: 0.5000, Validation Loss: 0.2500 on Train Data
Epoch 2/5, Loss: 0.5000, Validation Loss: 0.0000 on Train Data
Epoch 3/5, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 4/5, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 5/5, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Training completed with Overall Loss 0.0000 on Train Data

Predictions:
[[0]
 [1]
 [1]
 [1]]

Last forward:
Neuron, inputs: [1 1], weights: [1.49 0.28], bias: 0.38, z: 2.15, activation: heaviside/step, a: 1


In [42]:
# Пример AND
# Настройка вывода для удобочитаемости
np.set_printoptions(precision=4, suppress=True, threshold=6, edgeitems=2, linewidth=80)

X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([[0],
              [0],
              [0],
              [1]])

X_train, y_train = X, y
X_test, y_test = X, y

neuron = Neuron(activation=heaviside, loss=MAE(), optimizer=SGD(learning_rate=1, momentum=0))
neuron.fit(X_train, y_train, epochs=10)

# y_pred = neuron.predict(X_test)
y_pred = neuron(X_test)
print(f'\nPredictions:\n{y_pred}')
print(f'\nLast forward:\n{neuron}')

Training started with Overall Loss 0.2500 on Train Data
Epoch 1/10, Loss: 0.2500, Validation Loss: 0.2500 on Train Data
Epoch 2/10, Loss: 0.2500, Validation Loss: 0.2500 on Train Data
Epoch 3/10, Loss: 0.5000, Validation Loss: 0.2500 on Train Data
Epoch 4/10, Loss: 0.5000, Validation Loss: 0.0000 on Train Data
Epoch 5/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 6/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 7/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 8/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 9/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Epoch 10/10, Loss: 0.0000, Validation Loss: 0.0000 on Train Data
Training completed with Overall Loss 0.0000 on Train Data

Predictions:
[[0]
 [0]
 [0]
 [1]]

Last forward:
Neuron, inputs: [1 1], weights: [1.05 1.02], bias: -1.18, z: 0.89, activation: heaviside/step, a: 1
